In [2]:
from kubernetes import client
from kfserving import KFServingClient
from kfserving import constants
from kfserving import utils
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2TensorflowSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kubernetes.client import V1ResourceRequirements
import os
import sys
import argparse
import logging

'''
'''
class KFServingOnNoteBook(object):
    def serving(self, storage_uri, namespace, serving_name):
        api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
        default_endpoint_spec = V1alpha2EndpointSpec(
                                  predictor=V1alpha2PredictorSpec(
                                    tensorflow=V1alpha2TensorflowSpec(
                                      storage_uri=storage_uri,
                                      resources=V1ResourceRequirements(
                                          requests={'cpu':'100m','memory':'1Gi'},
                                          limits={'cpu':'100m', 'memory':'1Gi'}))))
        isvc = V1alpha2InferenceService(api_version=api_version,
                                  kind=constants.KFSERVING_KIND,
                                  metadata=client.V1ObjectMeta(
                                      name=serving_name, namespace=namespace),
                                  spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec))        

        KFServing = KFServingClient()
        KFServing.create(isvc)

        KFServing.get(serving_name, namespace=namespace, watch=True, timeout_seconds=300)
        
if __name__ == '__main__':
    serving_service = KFServingOnNoteBook()
    serving_service.serving("pvc://fashion-mnist/saved_model", "dudaji", "fashion-mnist-server")
        

NAME                 READY      DEFAULT_TRAFFIC CANARY_TRAFFIC  URL                                               
fashion-mnist-server Unknown                                                                                      
fashion-mnist-server False                                                                                        
fashion-mnist-server False                                                                                        
fashion-mnist-server False                                                                                        
fashion-mnist-server True       100                             http://fashion-mnist-server.dudaji.example.com/...


In [6]:
!kubectl get inferenceservice

NAME                   URL                                                                             READY   DEFAULT TRAFFIC   CANARY TRAFFIC   AGE
fashion-mnist-server   http://fashion-mnist-server.dudaji.example.com/v1/models/fashion-mnist-server   True    100                                51m


In [3]:
!kubectl get svc

NAME                                                 TYPE           CLUSTER-IP       EXTERNAL-IP                                           PORT(S)             AGE
fashion-mnist                                        ClusterIP      10.97.254.46     <none>                                                80/TCP              16h
fashion-mnist-server-predictor-default               ExternalName   <none>           istio-ingressgateway.istio-system.svc.cluster.local   <none>              12m
fashion-mnist-server-predictor-default-mpw4f         ClusterIP      10.106.86.184    <none>                                                80/TCP              12m
fashion-mnist-server-predictor-default-mpw4f-shn5p   ClusterIP      10.109.15.117    <none>                                                80/TCP              12m
fashion-mnist-server-predictor-default-mpw4f-xfhbg   ClusterIP      10.103.188.45    <none>                                                9090/TCP,9091/TCP   12m
lab-test              

In [5]:
!saved_model_cli show --dir ./saved_model/001 --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['flatten_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28)
      name: serving_default_flatten_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict
